In [15]:
import sys
from os.path import pardir, sep
import pandas as pd
import random
import numpy as np
import torch
import os
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics import Accuracy
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl
from dataset import DataPreprocUtils
from model import LSTMNet
import wandb
from datetime import datetime
import argparse
import logging
from gensim.models import fasttext
import numpy as np
import pathlib
import joblib


sys.path.append("/home/P76114511/IKM-nlp-practice/AGTC")
from model import LSTMNet
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [22]:

ROOT = f'/home/P76114511/IKM-nlp-practice'
PATH = {
    'no_pretrained':f'{ROOT}/AGTC/checkpoints/AG-epoch=09-val_acc=0.90.ckpt',
    'pretrained':f'{ROOT}/AGTC/pretrained_checkpoints/AG-epoch=09-val_acc=0.92.ckpt'
}
NPE = LSTMNet.load_from_checkpoint(PATH['no_pretrained'])
preds = []
golds = []
NPE.eval()
# testloader needs to be pickled... 
# not a very beautiful way of doing it but it's so late I want to sleep

testloader = joblib.load('/home/P76114511/IKM-nlp-practice/data/AG-testloader.pkl')
for X, y in testloader:
    with torch.no_grad():
        y_hat = NPE(X)
        y_hat = torch.argmax(y_hat, axis = 1)
    golds.extend(y.numpy())
    preds.extend(y_hat.numpy())


Global seed set to 42


In [23]:
print(accuracy_score(golds, preds))
npestats = precision_recall_fscore_support(golds, preds, average='macro')

0.9028947368421053


In [24]:

PE = LSTMNet.load_from_checkpoint(PATH['pretrained'])
preds = []
golds = []
PE.eval()

for X, y in testloader:
    with torch.no_grad():
        y_hat = PE(X)
        y_hat = torch.argmax(y_hat, axis = 1)
    golds.extend(y.numpy())
    preds.extend(y_hat.numpy())
print(accuracy_score(golds, preds))
pestats = precision_recall_fscore_support(golds, preds, average='macro')


Global seed set to 42


0.9227631578947368


In [27]:
# w/o https://wandb.ai/nana2929/IKM-nlp-practice/runs/1o26tdly
# w/ https://wandb.ai/nana2929/IKM-nlp-practice/runs/3qo3tj38
pd.DataFrame({'w/o pretrained': npestats, 
            'w/ pretrained': pestats}, index = ['precision', 'recall', 'f1 macro', 'support'])

,w/o pretrained,w/ pretrained
precision,0.903494,0.923097
recall,0.902895,0.922763
f1 macro,0.902972,0.922768
support,NaN,NaN
